<a href="https://colab.research.google.com/github/bethmiecz/Question_Answering_SQuAD2.0/blob/main/Question_Answering_on_SQuAD_2_0_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question Answering on Squad**



In [ ]:
!wget https://p4-squad-data.s3.us-east-2.amazonaws.com/train-v2.0.json

--2020-12-22 01:28:00--  https://p4-squad-data.s3.us-east-2.amazonaws.com/train-v2.0.json
Resolving p4-squad-data.s3.us-east-2.amazonaws.com (p4-squad-data.s3.us-east-2.amazonaws.com)... 52.219.101.226
Connecting to p4-squad-data.s3.us-east-2.amazonaws.com (p4-squad-data.s3.us-east-2.amazonaws.com)|52.219.101.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  72.6MB/s    in 0.6s    

2020-12-22 01:28:01 (72.6 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [ ]:
!pip install transformers

In [ ]:
import collections
import json
import string
import re

import numpy as np
import transformers
import nltk 
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk.classify.maxent 
import spacy

In [ ]:
class Answer:
  def __init__(self, text, answer_start):
    self.text = text
    self.answer_start = answer_start

class Question:
  def __init__(self, question, id, answers, is_impossible, par_id):
    self.question = question
    self.id = id
    self.answers = answers 
    self.is_impossible = is_impossible
    self.par_id = par_id

class Paragraph:
  def __init__(self, questions, context, id):
    self.questions = questions
    self.context = context
    self.id = id

class Text:
  def __init__(self, title, paragraphs):
    self.title = title
    self.paragraphs = paragraphs


In [ ]:
# PREPROCESSING AND DATA EXPLORATION

import random as random

train_set, validation_set, all_text, unanswerable = [[] for _ in range(4)]
total_questions, max_questions, total_questions, max_answer, total_pars, max_pars, max_len_q, total_answerable, total_unanswerable, index = [0 for _ in range(10)]

with open('train-v2.0.json') as json_file:
  json1 = json.load(json_file)
  jdata = json1['data']

  for item in jdata:
    par_info = item['paragraphs']
    paragraphs = []

    for par_num in range(0, len(par_info)):
      par = par_info[par_num]
      questions = []

      for qa in par['qas']:
        answers = []

        for a in qa['answers']:
          answer = Answer(a['text'], a['answer_start']) 
          answers.append(answer)
        if (qa['is_impossible'] == True):
          answer = Answer("No answer", 0) 
          answers.append(answer)
        q = Question(qa['question'], qa['id'], answers, qa['is_impossible'], par_num)
        questions.append(q)
        index += 1
        if (len(answers) > max_answer):
          max_answer = len(answers)
        if (qa['is_impossible'] == False):
          total_answerable += 1
        else:
          total_unanswerable += 1
          unanswerable.append(q)
        if (len(q.question.split(" ")) >= max_len_q):
          max_len_q = len(q.question.split(" "))

        # Splitting into validation and training randomly.
        if random.randint(1,20) is 1: 
          validation_set.append(q)
        elif random.randint(1,10) is 2:
          train_set.append(q)

      paragraph = Paragraph(questions, par['context'], par_num)
      total_questions += len(questions)
      if (len(questions) > max_questions):
        max_questions = len(questions)
      paragraphs.append(paragraph) 

    total_pars += len(paragraphs)
    if (len(paragraphs) > max_pars):
      max_pars = len(paragraphs)
    d = Text(item['title'], paragraphs)
    all_text.append(d)

In [ ]:
# Decreasing size of training set for initial testing (comment out for final run). 

quarter_of_train = int(len(train_set)*.25)
#training_set = train_set[:quarter_of_train]
training_set = train_set

### Initial data observations

In [ ]:
print("There are " + str(len(all_text)) + " total blocks of text, each of which has a title, set of questions with answers, and a context.")
print(str(len(training_set)) + " of these are used for training, and " + str(len(validation_set)) + " for validation.")

print("")
print("There are " + str(total_questions) + " total questions.")
print("The largest number of questions per paragraphs is " + str(max_questions) + ".")
print("The largest number of answers per question is " + str(max_answer) + ", so each question has 0 or 1 answer. ")
print("There are " + str(total_answerable) + " questions with an answer, and " + str(total_unanswerable) + " that are unanswerable.")
print("That means " + str(total_unanswerable / (total_answerable + total_unanswerable) * 100) + "% of the questions are unanswerable.")
print("There are approximately 2x more answerable questions than unanswerable ones.")
print("")
print("Here are some examples of unanswerable questions:")
print(unanswerable[0].question)
print(unanswerable[5000].question)
print(unanswerable[1001].question)

print("")
print("There are " + str(total_pars) + " total paragraphs.")
print("The largest number of paragraphs per article is " + str(max_pars) + ".")

print("")
print("There are an average of " + str(int(total_pars / len(all_text))) + " paragraphs per block of text.")
print("There are an average of " + str(int(total_questions / total_pars)) + " questions per paragraph.")

print("")
print("Here is an example of a question and answer:")
print("Title of Text: " + all_text[10].title)
print("Question: " + all_text[10].paragraphs[3].questions[2].question)
print("Question ID: " + all_text[10].paragraphs[3].questions[2].id)
print("Is Impossible: " + str(all_text[10].paragraphs[3].questions[2].is_impossible))
print("Answer: " + all_text[10].paragraphs[3].questions[2].answers[0].text)
print("Answer Start: " + str(all_text[10].paragraphs[3].questions[2].answers[0].answer_start))
print("Context: " + all_text[10].paragraphs[3].context)

There are 442 total blocks of text, each of which has a title, set of questions with answers, and a context.
12433 of these are used for training, and 6498 for validation.

There are 130319 total questions.
The largest number of questions per paragraphs is 30.
The largest number of answers per question is 1, so each question has 0 or 1 answer. 
There are 86821 questions with an answer, and 43498 that are unanswerable.
That means 33.37809528925176% of the questions are unanswerable.
There are approximately 2x more answerable questions than unanswerable ones.

Here are some examples of unanswerable questions:
What category of game is Legend of Zelda: Australia Twilight?
What is never necessary to make any changes to either document which governs the operations of the Royal Institute?
Who sued Internet Archive in 2004?

There are 19035 total paragraphs.
The largest number of paragraphs per article is 149.

There are an average of 43 paragraphs per block of text.
There are an average of 6 

Here are some of my initial quantitative observations about the dataset, along with an example of a question, context, and answer. 

This is a very large dataset in total. Due to RAM continually crashing, I have decided to only use 1/5 of it for training and 1/10 for validation. This is not ideal, but due to the constraints of Colab I have no choice. Therefore, I will have approximately 100,000 questions, answers, and corresponding context information to train a model, and around 10,000-15,000 for validation. 

Based on these initial observations, I can already predict some of the issues that may exist with processing this data and trying to output an answer. There are many characters that will have to be considered and normalized in the proper way, such as accents, punctuation, and separated pronunciations of names. Also, there are many more answerable questions than unanswerable ones, so the probability of that prediction will be lower in the model. 

# Metrics for Analysis

$$F_1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$$

$$Precision = \frac{TP}{TP + FP} = \frac{\text{# of matching tokens}}{\text{# of predicted tokens}}$$

$$Recall = \frac{TP}{TP + FN} = \frac{\text{# of matching tokens}}{\text{# of gold tokens}}$$

$TP$ refers to the number of true positives; $FP$ is the number of false positives; and $FN$ is the number of false negatives.


In [ ]:
# Some potentially useful helper functions for the metrics
def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def remove_punc_not_ques(text):
    remove = string.punctuation
    remove = remove.replace("?", "") 
    pattern = r"[{}]".format(remove)

    return ''.join(ch for ch in text if ch not in remove)

def normalize_keep_names(s):
  return white_space_fix(remove_articles(remove_punc(s)))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

In [ ]:
def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans):
    new_scores = {}
    return new_scores

def make_eval_dict(exact_scores, f1_scores, qid_list=None):
    return collections.OrderedDict([
        ("exact", numpy.mean(exact_scores)),
        ("f1", numpy.mean(f1_scores)),
        ("total", 0.),
    ])

def merge_eval(main_eval, new_eval, prefix):
    for k in new_eval:
        main_eval['%s_%s' % (prefix, k)] = new_eval[k]

In [ ]:
# This is used in my MEMM error analysis. 
exact_arr = []

In [ ]:
# Calculate the percentage of predictions that match any one of the ground truth answers exactly
def compute_exact(a_gold, a_pred):
  total_exact = 0

  for i in range(0, len(a_gold)):
    if a_gold[i] == a_pred[i]:
      total_exact += 1
  if len(a_pred) == 0:
    return 0
  return total_exact / len(a_pred)

# Calculate the f1 score for one array of predictions and another of correct outputs
def compute_f1(a_gold, a_pred):
  true_positives, precision, recall = [0 for _ in range(3)]
  for g in a_gold:
    for p in a_pred:
      if (g == p):
        true_positives += 1
  if len(a_pred) == 0:
    precision = 0
  else:
    precision = true_positives / len(a_pred)
  if len(a_gold) == 0:
    recall = 0
  else:
    recall = true_positives / len(a_gold)

  if (precision+recall == 0):
    f1 = 0.0
  else:
    f1 = (2*precision*recall)/(precision + recall)
    
  return f1


In [ ]:
# This functions iterates through all of the questions in a training or validation set.
# It looks for a question as a key in predictions and correct answers, then calculates
# exact match and f1 accordingly. 

def compute_exact_and_f1(all_gold, all_pred, set):
  exact = 0
  f1 = 0
  len = 0
  for question in set:
    if question.question in all_pred:
      prediction = all_pred[question.question]
    else:
      prediction = ""
    if question.question in all_gold:
      std = all_gold[question.question]
    else:
      std = ""
    if (question.is_impossible and prediction == ""):
      exact += 1
      f1 += 1
      exact_arr.append(question)
    else:
      exact += compute_exact(std, prediction)
      f1 += compute_f1(std, prediction)
    len += 1
  if len == 0:
    return 0
  return exact/len, f1/len


In [ ]:
# testing exact match and f1
a_gold = ["The blue is sky"]
a_pred = ["The sky is blue"]

print(compute_exact(a_gold, a_pred))
print(compute_f1(a_gold, a_pred))

0.0
0.0


In [ ]:
# MEMM IMPLEMENTATION:

# 1. Extract Features
# 2. Train MaxEnt
# 3. Call Viterbi 

In [ ]:
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Retrieve paragraph from specific question. 
def get_par_from_question(q):
  par_num = q.par_id
  for t in all_text:
    for p in t.paragraphs:
      if p.id == par_num:
        return p
  return 0

In [ ]:
# PREPROCESSING (continued): tried many different ways of normalizing data. 
# Final design decision = remove punctuation but keep uppercase letters.
# Without uppercase letters, named-entity recognition and POS tagging fail for proper nouns. 

for q in (training_set+validation_set):
  q.question = remove_punc(q.question)
  if (not q.is_impossible):
    q.answers[0].text = remove_punc(q.answers[0].text)
  p = get_par_from_question(q)
  p.context = remove_punc(p.context)

In [ ]:
# Convert answer_start from a character to a token index in context. 
def get_answer_span(q, c):
  # Find new answer_start after normalizing data (different index once punctuation is removed). 
  char_start = c.find(q.answers[0].text) 
  char_end = char_start + len(q.answers[0].text)
  if (q.is_impossible):
    return 0,0
  token_num, start_token, end_token = [0 for _ in range(3)]
  for i in range(0, len(c)):
    if c[i] == ' ':
      token_num += 1
    if i == char_start:
      start_token = token_num
    elif i == char_end:
      end_token = token_num
  return start_token, end_token

In [ ]:
# Testing get_answer_span after normalizing data. 
s, e = get_answer_span(training_set[0], get_par_from_question(training_set[0]).context)
print("Correct answer is: " + training_set[0].answers[0].text)
print("Output from get_par is: " + str(get_par_from_question(training_set[1]).context.split()[s:e]))

Correct answer is: Dangerously in Love
Output from get_par is: ['Dangerously', 'in', 'Love']


In [ ]:
# TOKENIZATION: ANSWERABILITY AND ANSWER SPAN

# Function to split questions and corresponding context into tokens and labels.
# Labels = 0 if part of question, 1 if context but not answer, 2 if answer. 

def split_up_tokens(set, all_ne):
  context_question_tokenized, ne_tokens = [{} for _ in range(2)]
  context_question_labels, context_question_tokens = [[] for _ in range(2)]

  for qs in set:
    par = get_par_from_question(qs)
    c = par.context
    context = c.split()
    q = qs.question

    # Iterate through entire question, and append a label 0. 
    for word in q.split():
      if word in context_question_tokenized.keys():
        context_question_tokenized[word].append(0)
      else:
        context_question_tokenized[word] = [0]
      context_question_labels.append(0)
      context_question_tokens.append(word)

    answer_start = len(context)

    if (not(qs.is_impossible)):
      answer_start, answer_stop = get_answer_span(qs, c)

    # For answer span, append all 1s before answer. 
    for word in context[:answer_start]:
      if word in context_question_tokenized.keys():
        context_question_tokenized[word].append(1)
      else:
        context_question_tokenized[word] = [1]
      context_question_labels.append(1)
      context_question_tokens.append(word)
    if (not(qs.is_impossible)):

      # Append all 2s for answer span. 
      for word in context[answer_start:answer_stop]:
        if word in context_question_tokenized.keys():
          context_question_tokenized[word].append(2)
        else:
          context_question_tokenized[word] = [2]
        context_question_labels.append(2)
        context_question_tokens.append(word)

      # Append 1s again for context after answer span. 
      for word in context[answer_stop:]:
        if word in context_question_tokenized.keys():
          context_question_tokenized[word].append(1)
        else:
          context_question_tokenized[word] = [1]
        context_question_labels.append(1)
        context_question_tokens.append(word)
    
    # Named-entity recognition for each token using SPACY (only calculate on first iteration of all words in vocab).
    if all_ne:
      cq = q+c 
      t = nlp(cq)
      for ent in t.ents:
        ne_tokens[ent.text] = ent.label_

  return context_question_tokenized, context_question_tokens, context_question_labels, ne_tokens

In [ ]:
# Tokenize all training and validation data to create vocabulary for data, and generate named entities for vocab. 
context_question_tokenized, context_question_tokens, context_question_labels, ne_tokens = split_up_tokens(training_set+validation_set, True)

In [ ]:
# SPACY for some reason thinks 'Kanye' is a geographical location, so I am hardcoding it here. 
ne_tokens['Kanye'] = "PERSON"

In [ ]:
# Testing tokenization labels are 0 for question, 1 for words in context not in answer span, 2 for answer span. 
print(context_question_labels[30:50])
print(context_question_tokens[30:50])
print(context_question_tokenized['Kanye'])
print(ne_tokens['Houston'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['and', 'raised', 'in', 'Houston', 'Texas', 'she', 'performed', 'in', 'various', 'singing', 'and', 'dancing', 'competitions', 'as', 'a', 'child', 'and', 'rose', 'to', 'fame']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
# BIGRAM CREATION
bigram_dictionary = {}
bigram_lst = []

for i in range(0, len(context_question_labels)):
  if i is not 0:
    bigram = (context_question_tokens[i-1], context_question_tokens[i])
    label = [(context_question_labels[i-1], context_question_labels[i])]
    bigram_lst.append(label)
    if bigram not in bigram_dictionary:
       bigram_dictionary[bigram] = label
    else:
       bigram_dictionary[bigram].append(label)

In [ ]:
# UNKNOWN WORD HANDLING AND SMOOTHING
def handle_unk(word_counts):
  k = int(499 * len(word_counts) / 500)
  items = sorted(word_counts, key=lambda key: len(word_counts[key]), reverse=True)
  topk = {}
  bottomk = {}
  for i, item in enumerate(items):
    if i < k:
      topk[item] = word_counts[item]
    else:
      bottomk[item] = word_counts[item]
  
  total = 0 
  ones = 0
  for word in bottomk: 
    total += len(bottomk[word])
    ones += np.sum(bottomk[word])
  unk_values = []
  unk_values.extend([2 for _ in range(int(total/4))])
  topk["<UNK>"] = unk_values
  return topk

In [ ]:
word_tokens_unk = handle_unk(context_question_tokenized)

In [ ]:
# Tokenize training and validation separately, but without re-running named entity creation. 
trtokenized, train_cqtokens, train_cqlabels, train_namedents = split_up_tokens(training_set, False)
valtokenized, val_cqtokens, val_cqlabels, val_namedents = split_up_tokens(validation_set, False)

In [ ]:
# BIGRAM HANDLING
count_00, count_01, count_10, count_11, count_02, count_20, count_12, count_21, count_22 = [0 for _ in range(9)]

for bigram in bigram_dictionary.keys():
  labels_lst = bigram_dictionary.get(bigram)
  tot_bigram_count = len(labels_lst)
  for lab in labels_lst:
    if lab == [(0, 0)]:
      count_00 += 1
    elif lab == [(0, 1)]:
      count_01 += 1
    elif lab == [(1, 0)]:
      count_10 += 1
    elif lab == [(1, 1)]:
      count_11 += 1
    elif lab == [(0, 2)]:
      count_02 += 1
    elif lab == [(2, 0)]:
      count_20 += 1
    elif lab == [(1, 2)]:
      count_12 += 1
    elif lab == [(2, 1)]:
      count_21 += 1
    elif lab == [(2, 2)]:
      count_22 += 1

In [ ]:
# TRANSMISSION PROBABILITY MATRIX. 
bigram_p = 0
bigram_p00 = count_00 / (count_00 + count_01 + count_02)
bigram_p11 = count_11 / (count_11 + count_10 + count_12)
bigram_p01 = count_01 / (count_00 + count_01 + count_02)
bigram_p10 = count_10 / (count_11 + count_10 + count_12)
bigram_p02 = count_02 / (count_00 + count_01 + count_02)
bigram_p20 = count_20 / (count_21 + count_20 + count_22)
bigram_p12 = count_12 / (count_11 + count_10 + count_12)
bigram_p21 = count_21 / (count_21 + count_20 + count_22)
bigram_p22 = count_22 / (count_21 + count_20 + count_22)

trans_p = [[bigram_p00, bigram_p01, bigram_p02], [bigram_p10, bigram_p11, bigram_p12], [bigram_p20, bigram_p21, bigram_p22]]
  
# EMISSION PROBABILITY MATRIX. 
emiss_p, q_lst, not_a_lst, a_lst = [[] for _ in range(4)]
word_to_idx = {}
idx_to_word = {}
for i, tok in enumerate(word_tokens_unk):
  word_to_idx[tok] = i
  idx_to_word[i] = tok
  tags = word_tokens_unk.get(tok)
  total_tags = len(tags)
  q_tag = tags.count(0) + 100
  not_a_tag = tags.count(1) 
  a_tag = tags.count(2) + 1000000000
      
  q_p = q_tag / (len(word_tokens_unk)*100 + total_tags)
  not_a_p = not_a_tag / (len(word_tokens_unk) + total_tags)
  a_p = a_tag / (len(word_tokens_unk)*1000000000 + total_tags)

  q_lst.append(q_p)
  not_a_lst.append(not_a_p)
  a_lst.append(a_p)

emiss_p.append(q_lst)
emiss_p.append(not_a_lst)
emiss_p.append(a_lst)

In [ ]:
# PART-OF-SPEECH TAGGING + TEST
pos_tags = nltk.pos_tag(word_tokens_unk)
print(pos_tags[word_to_idx['Houston']])

('Houston', 'NNP')


In [ ]:
# ADDITIONAL NAMED-ENTITY TAGGING.
# I decided to assign no entity to unknown words, and assign entities to question words to match question-type to answer-type.
# For example, "Who" in question indicates need to find a word with PERSON label in context. 
ne_tokens['<UNK>'] = ""
ne_tokens['Who'] = "PERSON"
ne_tokens['When'] = "DATE"
ne_tokens["Where"] = "GPE"
ne_tokens["What"] = ""
ne_tokens["In"] = "DATE"
ne_tokens["Which"] = ""

In [ ]:
# VITERBI ALGORITHM: search for most likely sequence 
# given original probabilities for each state, transition probabilities, and emission probabilities. 

def viterbi(doc, init_p, trans_p, classifier):

  n = len(doc)
  c = len(init_p) # Number of categories = 3
  score = np.zeros((c,n))
  bptr = np.zeros((c,n), dtype=np.int32)
  tags = [0 for _ in range(n)]

  for i in range(0, c):
    idx = word_to_idx[doc[0] if doc[0] in word_tokens_unk else '<UNK>']
    dictionary = {'word':doc[0] if doc[0] in word_to_idx else "<UNK>", 'pos':pos_tags[idx][1]}
    score[i][0] = np.log(classifier.prob_classify(dictionary).prob(i))
    bptr[i][0] = 0 
  for t in range(len(doc)):
    for i in range(0,c):
      idx = word_to_idx[doc[t] if doc[t] in word_tokens_unk else '<UNK>']
      dictionary = {'word':doc[t] if doc[t] in word_to_idx else "<UNK>", 'pos':pos_tags[idx][1]}
      vals = [score[j][t-1] + np.log(trans_p[j][i]) for j in range(0, c)]

      score[i][t] = max(vals) + np.log(\
        classifier.prob_classify(dictionary).prob(i))
      bptr[i][t] = np.argmax(vals) 

  tags = np.zeros(n, dtype=np.int32)
  tags[n - 1] = np.argmax(score[0:2,n-1])
  for k in reversed(range(n-1)):
    tags[k] = bptr[tags[k+1] , k+1]

  return tags

In [ ]:
# CREATING DICTIONARY FOR ANSWER SPAN MEMM 
features_pos = [] 
for i, token in enumerate(train_cqtokens):
  features_pos.append(({'word':token, 
                        'pos':pos_tags[word_to_idx[token if token in word_tokens_unk else '<UNK>']][1],
                        'type': ne_tokens[token if token in ne_tokens.keys() else '<UNK>']}, 
                       train_cqlabels[i]))

In [ ]:
# TRAINING MEMM FOR ANSWER SPAN CLASSIFICATION
pos_classifier = nltk.classify.MaxentClassifier.train(features_pos, max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.013
             2          -0.12871        0.928
             3          -0.12431        0.930
             4          -0.11941        0.933
         Final          -0.11548        0.942


In [ ]:
print(pos_classifier.prob_classify({'word': '2003', 'pos': 'NN', 'type': 'DATE'}).prob(2))

In [ ]:
# Function that outputs the predicted answer span. 

def create_output_answer(classifier, set):
  answers = []
  for i, question in enumerate(set):
    potential_answers = []
    par = get_par_from_question(question)
    q = question.question
    tot = q+par.context
    # Go through Viterbi and pull out potential answers. 
    viterbi_training = viterbi(tot, init_p, trans_p, classifier)
    for j, output in enumerate(viterbi_training):
      if output == 2:
        if viterbi_training[j-1] == 2:
          potential_answers[len(potential_answers)-1] = potential_answers[len(potential_answers)-1] + " " + output
        else:
          potential_answers.append(output)
    prev = ""
    # Backtrack to see if the Viterbi missed any potential answers with a probability of 2 classification above threshold. 
    for i, word in enumerate(tot.split()):
      idx = word_to_idx[word if word in word_tokens_unk else '<UNK>']
      p2 = pos_classifier.prob_classify({'word': word, 'pos': pos_tags[idx][1]}).prob(2)
      if (p2 > 0.04):
        if tot.split()[i-1] == prev:
          potential_answers[len(potential_answers)-1] = potential_answers[len(potential_answers)-1] + " " + word
        else:
          potential_answers.append(word)
        prev = word
    # Generating the answer span by looking through all potential answers and seeing which matches question type with named entity. 
    if (len(potential_answers) > 1):
      firstword = tot.split(" ", 1)[0]
      if firstword in ne_tokens.keys():
        question_type = ne_tokens[firstword]
      else:
        question_type = ""
      answer = ""
      for pot in potential_answers:
        for token in pot:
          type = ""
          if token in ne_tokens.keys():
            type = ne_tokens[token]
          if type == question_type:
            answer = pot
      else:
        answer = potential_answers[0]
    answers.append(answer)
  
  return answers


In [ ]:
# Last attempt to weight the words with label 1 lower. 
init_p = [0.4, 0.2, 0.4]

training_answers = create_output_answer(pos_classifier, training_set)

**Summary:**
I decided to implement a Maximum Entropy Markov Model trained to detect question-context-answer labels, part-of-speech, and named entities personalized specifically for SQuAD 2.0. 

**Steps:**
Next, I will break down the steps for creating this twofold MEMM:
1. *Preprocessing the dataset:* I decided the best way to utilize the data was to create classes for Answer, Question, Paragraph, and Text. Every instance of Text contains a title and an array of paragraphs. Every Paragraph contains an id which I created to easily find a question's corresponding article, context, and an array of questions. Every Question has a question text, id, answer (if answerable), a boolean for whether it has an answer or not, and a par_id to match the paragraph it belongs to. Every question is randomly assorted into the training or validation set- I decided to create these sets by question rather than paragraph so that there would be questions from every text/context in both sets. 
2. *Normalizing the data:* I tried multiple ways of normalizing all of the text to best fit the task:
*   Removing articles made it easier to pull out important words, but also caused the system to fail to produce exact matches between the correct answer and the output. Many of the correct answers in the training data contain articles such as "the" or "an". 
*   Converting all of the text to lowercase also caused the system to fail. Once all of the words were converted to lowercase, part-of-speech tagging failed to differentiate between proper nouns, PNN, and nouns, NN. This created a tremendous problem because for questions such as "who?", I need to differentiate between names and non-names when pulling out potential answers. 
*   Removing all punctuation except for question marks made the system better at differentiating between words in the question and words in the context, but ultimately left some question marks in the context itself which then muddled the semantics of the sentences. It also made it difficult to pull out the correct answer span once all the indices were changed. 
*   Removing all punctuation was my ultimate design decision. I kept case as it is for all words so the system can properly distinguish words like names and locations, and I kept articles for the final output to be a (hopeful) exact match to the correct answer. I removed all punctuation to improve POS and named-entity recognition. For example, in the question "When did Beyonce leave Destinys Child and become a solo singer?", the correct answer is 2003. However, the system would never have outputted 2003 because in the context it appears as '(2003)'. The parentheses messed up SPACY named-entity recognition and caused the label to be "" instead of a "number". 
3. *Finding correct answer span:* It was difficult to find the correct indices of the answer span once the data was normalized, since the original answer_start is in terms of characters instead of tokens, and now all punctuation characters were removed. I created a function to find the starting character index of a substring (the correct answer span) in the context, in order to convert to token indices of the answer in the context. I then tested this function on an example from the training dataset. 
3. *Tokenization & Named-Entity Labelling:* I iterate through every question in the dataset, and pull out its corresponding context. I decided to create a dictionary that maps every word in the vocabulary to an array of labels it's been assigned to, as well as two ordered arrays of every single token passed through the function and its assigned label. For answer span, I used 3 labels, where a token was assigned 0 if part of the question, and then words in the context were assigned either 1 if part of the context but not the answer span and 2 if part of the answer span. Here is also where I performed named-entity recognition on every concatenated question+context and used a dictionary to map a token to its assigned ne label. I unfortunately discovered that SPACY for some reason thinks the word "Kanye" is a geographical location, and nltk thinks the word "Kanye" is a verb. Since the word "Kanye" appears so often in the training and validation data, I hardcoded the correct named-entity and POS for this token.
5. *Unknown Word Handling and Smoothing:* I used a very similar approach to the second project, where I mapped indices to tokens in one data structure, and then pull out those indices to find the top-k most-used words. Then, I mapped those indices to word tokens and add them to a data structure, count up the number of tokens left out, assign them to <UNK>, and add 2s to that <UNK> to make up for the bottom-k values. I decided to add 2s to these unknowns because the answer-span label, 2, is the one which occurs the least in the dataset and had a very low potential output, since the answer phrase is typically much much smaller than the total context/paragraph.
6. *Bigram Assignment & Transmission Matrix:* Similarly to project 2, I created a dictionary of every bigram label in the dataset. However, there are 9 possible bigrams instead of the 4 in P2 since there are 3 possible labels. I count up all of the occurrences of each bigram type to calculate the probability of each and input them into the transmission matrix. 
7. *Add-k Smoothing & Emission Matrix:* I also performed add-k smoothing on the data. I ultimately decided on different k-values for question (0), non-answer context (1), and answer (2) since they occur in such differing spans in the data. I weighted answer labels the highest, then question labels, since the size of the context is significantly larger than either for every single question/corresponding paragraph. In order to ensure that this wouldn't cause some sort of error in calculating probabilities, I made sure that the newly smoothed values add up to approximately 1 (closer to 0.98) for each word's likelihood of being assigned a 0, 1, or 2. I then used the probability that a token is a question, not-answer, or answer and created the emission matrix. 
8. *Part-of-Speech Tagging:* I used nltk's built-in POS classification to classify the part of speech for every token in the vocabulary. 
9. *MEMM Feature Dictionaries:* I used the question-not answer-answer labels, the part of speech, and the named-entity tags and mapped each feature to a token to create a dictionary. I also performed additional named-entity tagging personalized for this task. On top of the built-in SPACY library, I decided to assign question words to the ne label that they correspond to. For example, I assigned the word "who", which previously had no label, to the label "PERSON" in the hopes this will improve the MaxEnt classifier's ability to match a question to its answer span type. 
10. *MEMM Training & Viterbi Algorithm:* I created one classifier answer-span detection using the dictionaries described above. I previously was training two separate models, one for answerability and one for detecting the actual answer span, but this was causing RAM to crash even on only 10% of the data. I trained on nltk's built-in MaxEntClassifier. I then created a Viterbi algorithm which takes into account the 3 types of labels (instead of 2 like for project 2), and returns the predicted sequence output for every concatenated question + its corresponding context. I run the entire output through the Viterbi. I then run through all the tokens not in the potential answer with classification probability of outputting a 2 (meaning in answer) greater than a threshold of 3%. This is to see if any potential answers were left out of the Viterbi output. Then, I do one last check, where if there is more than one potential answer I look to see if the question type matches the named-entity of the output and use that one. This outputs the final answer phrase/word. 



*Why MaxEnt Classifier:*

The task for this project was twofold: 1. decide whether or not a question is *answerable* given an article of text, and 2. if answerable, output the specific word or phrase from the article that contains the answer. This is closely related to IR-based factoid question-answering seen in lecture, but with an additional output of deciding if a question has no answer (slide 7, November 9 lecture). I considered numerous models when initially considering this project, and ultimately decided to use an MEMM. Multinomial logistic regressions (aka MaxEnt classifiers) are successful and efficient ways of performing multi-class classifications. In class, we've seen them used for part-of-speech tagging, named-entities, and binary classification tasks such as determining if an article is propaganda or not (Chapter 5, page 15). Although SQuAD 2.0 seems much broader than those tasks, it can ultimately be broken down into two discrete sequence classifications: 

We need a ternary sequence classification which outputs a prediction for every token. We can look at every question and corresponding context, and label every token as part of the question, the context that is not the answer, and the answer span. We then need to establish a threshold for whether or not the probability a certain span of tokens is the answer deems the question unanswerable. If the model detects no answer span, or detects an answer span but with a low probability, then it should output an empty string. 

*Why Unknown Word Handling:*

This is also discussed below. The SQuAD dataset is very broad, and the possibility of a word being in a question or context when it was not in the original vocabulary is very possible. I used a k-value of 499/500 because the dataset is so large, and this value seemed to work the best for outputting a solid number of <UNK> tokens without overwhelmingly deleting some of the words with the lowest occurrences in the current vocabulary. 

*Why Add-K Smoothing:*
The need for add-k smoothing is clear in this instance. Let's take a look at the nature of every input into the Markov Model. In every input, we have a question and the entire corresponding context, with labels for question answer and not-answer. The probability of the model outputting a prediction of 2, aka answer span, is extremely low without this smoothing, causing it to basically never occur. This is because the number of tokens in the context with the label of 1 greatly outweight both the number of tokens in the question and the number in the answer span. I ultimately chose a values 10000000 for answer (2) and 100000 for question (0).  Any value lower than that for 2 caused an output that never predicts a single word as an answer. That k-value for 0 also created the best likelihood of predicting a word is a question correctly. 

*Why POS Tagging:*

I believe part-of-speech tagging is a useful feature for this task, and was a good starting point. nltk POS tagging distinguishes between proper nouns (PNNs) and regular nouns (NNs), which is especially helpful for question-answering tasks. The model needs to detect the types of words that should be predicted as an answer span, and some types of questions such as "who" or "where" will oftentimes need a proper noun somewhere in the answer. For example, the question "Where was Beyonce born?" should search through the context for a proper noun and a location, i.e. "Houston, Texas". POS tagging is also useful because oftentimes for this IR-based factoid question-answering, the answer does not contain a verb because it should be just a short phrase or single word which contains information. In the examples from lecture, none of the responses contained a verb. Therefore, POS-tagging will be useful in determining where to start and stop an answer span without grabbing too much information, which typically contains just nouns and qualifiers (slide 7, November 9). 

*Why Named-Entity Recognition:*

Part-of-speech tagging is a good stepping stone for finding an answer span, but named-entity recognition is the best way to fully accomplish this question-answering task. Many efficient QA systems make use of answer-type classification using named-entity classification. For example, the question "Who founded Virgin Airlines?" should be labelled as a question which needs a "PERSON" answer (slide 12, November 9). There are many methods of answer type taxonomy, but I ultimately decided on SPACY's built-in linguistic features and its named entity labelling system (https://spacy.io/api/annotation#named-entities, https://spacy.io/usage/linguistic-features). This library contains labels such as PERSON, GPE (country, city, state), LOC (other locations), DATE, and TIME which will be useful for 'who', 'where', and 'when' questions respectively. This type of named entity recognition is key for matching a question type to the type of words that should be pulled out as an answer from the context. 

## Validation Step



In [ ]:
# Answer sequences for validation set outputted using classifier and Viterbi. 
val_answers = create_output_answer(pos_classifier, validation_set)

In [ ]:
# Creating a dictionary of predicted outputs to questions. 
val_answers_pred = {}
for i, q in enumerate(validation_set):
  if q.question not in val_final_answers.keys():
    val_answers_pred[q.question] = val_answers[i]

In [ ]:
# Creating a dictionary of correct outputs to questions. 
val_gold = {}
for i, q in enumerate(validation_set):
  if q.question not in val_correct_answers.keys():
    if (q.is_impossible):
      val_gold[q.question] = ""
    else:
      val_gold[q.question] = q.answers[0].text

In [ ]:
exact, f1 = compute_exact_and_f1(val_gold, val_answers_pred, validation_set)

(0.33614077304857876, 0.33614077304857876)

We can see that the exact scores and f1 scores are the same, approximately 34%. I discuss why this is below. 

In [ ]:
print(len(exact_arr))

2235


In [ ]:
print(exact_arr[0].question)
print(exact_arr[0].answers[0].text)

What consoles can be used to play Australia Twilight
No answer


In [ ]:
print(exact_arr[10].question)
print(exact_arr[10].answers[0].text)

Sam Mendes wrote what 2015 film
No answer


In [ ]:
print(validation_set[6].question)
print(val_answers[6])
print(validation_set[6].answers[0].text)

In [ ]:
print(val_answers[18])
print(validation_set[18].answers[0].text)

Which Amy Winehouse
Back to Black


In [ ]:
print(validation_set[500].question)
print(val_answers[500])
print(validation_set[500].answers[0].text)

### Results Analysis

I evaluated the model by calculating exact match and f1 scores for a quantitative analysis, and then examining examples of correct and incorrect outputs for qualitative analysis. I also kept track of which features/design decisions produced the best results. 

1. My model has an f1 score and exact match score of approximately 34% for the chosen validation data. Upon further examination, I have discovered that most of the correct answers are the ones which are predicting "No answer", i.e. that the question is unanswerable given the context. Even without training a separate MEMM specifically for answerability, setting a threshold for probabilities under which the answer span would be deemed impossible seems to work fairly accurately for the model. It makes sense here that exact match and f1 are approximately the same, since all of the correct outputs are exactly correct (unanswerable) and the others have little overlap with the correct output. 
2. I examine the qualitative evaluation examples below, but in summary the system is successful in outputting an entity of the correct type (i.e. person for "who", location for "where"), but fails to pull out the correct answer sequence because it is stuck on weighting the question tokens too highly. 
3. I performed various attempts at designs and had varying success for each:
- My first try was creating a system with two discrete models, one for detecting a binary yes/no for answerability given a question and context, and the other pulling out the answer span. This system, given the inputs used for my current model, always predicted that something was answerable due to the greater percentage of that data in the training set. Even weighting those unanswerable values much higher (with k values of 100000 during add-k smoothing) did not change this result. 
- Any k-value during add-k smoothing for the label 2 (part of answer) below the current value led to no potential answers being outputted at all, i.e. leading the system to believe that every single question was unanswerable. 
4. In total, there are 2235 exactly correct outputs from the dataset, all of which are for unanswerable questions. 

My system works the best for determining whether or not a question is answerable given a context. It fails for answer span detection, but there are qualitative aspects of these outputs which I believe to be successful. 

I printed out some examples of correct and incorrect outputs above. For correct outputs, the model is successful with determining unanswerable questions, such as "What consoles can be used to play Australia Twilight?" and "Sam Mendes wrote what 2015 film?". 

The model is incorrect in determining the correct answer span when the question is answerable. For example:
- "What event occured after she was publicly criticized?" should output "boyfriend left her", but instead outputs "event occured". 
- "What Major League Soccer franchise is based in New York?" should output "New York City FC", but instead outputs "Major League Soccer".
- "What ancient religious document exists in two versions and includes Old Iranian language?" outputs "Tigers" instead of "Avestan".

Although these examples are obviously incorrect, they reveal useful information about the model and how it could be improved in the future. From these examples of incorrect outputs, we learn that:
1. Named entity recognition is working and useful. All of these outputs match the "type" of response which should be outputted. For example, a religious text and animal are both a "thing", and event and her boyfriend left are both occurrences. This means that POS and named-entity tagging are definitely successful features to be used for the model, thereby supporting my design decisions and feature choices/
2. The question tokens are being weighted too heavily. It is clear to see that the output is more likely to be pulled from the question than the correct answer span. That means the model knows the correct type it needs to pull out, such as a name or event, but struggles to differentiate between the context and question tokens. This can be improved in the future by weighing question tokens lower, and adding some sort of function where if a token is known to be in the question it cannot be outputted. 

I chose an unknown word handling value of 499/500. This is because this value produced the best results for my project 2 MEMM; I know that these are clearly different tasks, but I used many of the same features in my project 2, such as part-of-speech, and ultimately both systems can be reduced to a sequence tagging problem. Also, the dataset size was very similar for both projects (around 300,000 word tokens total for training and validation), since I had to use only 20% of the original SQuAD data or else RAM would crash. Since the dataset sizes are similar, this is basically a sequence classification, and some of the features overlap, I used the same k-value and unknown word handling method. 

Although unknown word handling is not always necessary for MEMM, it is helpful for this model. MaxEnt classifiers simply output a probability based on the inputted features, so they do not need to handle unknown words. However, this specific model relies heavily on named-entity tagging, and the named-entity assigned to an unknown word might be an empty string. This is because the input is being tokenized and entities are separated, so if the word is removed from its original context then its entity will disappear. Also, some words simply do not have SPACY label, such as if they are a thing or nondescript word. There are many potential words in question-answering not covered by the span of these paragraphs, questions, and answers. Therefore, I wanted to use unknown word handling to ensure that any unknown words would be given a named entity of "" to avoid mistakes in SPACY assignment. If an unknown word was given the wrong named entity tag, this could have tremendous implications for this system since it is so reliant on the probability of a word being matched to a question type or deemed unanswerable. 

# ALTERNATIVE: Implementing a Pre-trained Language Model

---

Next, I try utilizing BERT, or a Bidirectional Encoder Representations from Transformers. This is a partially pretrained maching learning library that uses an attention model. 

In [ ]:
!pip install tokenizers

In [ ]:
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast, TFBertModel, BertConfig

In [ ]:
# Uses BERT tokenizer to encode the context + question and concatenate them. 
def encode_c_q(c, q):
  c_tokenized = tokenizer.encode(c)
  q_tokenized = tokenizer.encode(q)
  return c_tokenized, q_tokenized

In [ ]:
# Generates 3 of the 5 required inputs. 
def get_ids(c_tok, q_tok):

  # 1. Input IDS, which are the concatenated tokens from encoding context and question. 
  input_ids = c_tok + q_tok

  # 2. Type IDs, which are 0 if the token is in context, 1 if in question. 
  type_ids = []
  for token in c_tok:
    type_ids.append(0)
  for token in q_tok:
    type_ids.append(1)

  # 3. Attention mask, which is a 1 for every token in the input. 
  attention_ids = []
  for id in input_ids:
     attention_ids.append(1)

  return input_ids, type_ids, attention_ids

In [ ]:
# Generates other 2 inputs to model. 
def get_indices(set):
  sindex, eindex = [[] for _ in range(2)]
  for ques in set:
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # 4. Starting index for every answer span. 
    sindex.append(start_idx)
    # 5. Ending index for every answer span. 
    eindex.append(end_idx)
  
  return sindex, eindex 

In [ ]:
def init_inputs(set):
  input_ids, type_ids, attention_mask = [[] for _ in range(3)]
  for ques in set:
    c = get_par_from_question(ques).context
    q = ques.question 
    c_tok, q_tok = encode_c_q(c, q)
    iids, tids, attmask = get_ids(c_tok, q_tok)

    lim = 500 - len(input_ids) # Design decision to set max length to 500
    if lim >= 0:
      for pad in range(0, lim):
        iids.append(0)
        tids.append(0)
        attmask.append(0)
    
    input_ids.append(iids)
    type_ids.append(tids)
    attention_mask.append(attmask)

  sindex, eindex = get_indices(set)
  return input_ids, attention_mask, type_ids, sindex, eindex

In [ ]:
import json
import random

def read_squad():
  with open('train-v2.0.json') as f:
    squad_dict = json.load(f)

    train_contexts, train_questions, train_answers, val_contexts, val_questions, val_answers = [[] for _ in range(6)]
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                answer = {}
                if (qa['is_impossible'] == True):
                  answer = {'text': '', 'answer_start': 0}
                else:
                  answer = qa['answers'][0]
                if random.randint(1,5) is 1: 
                  train_contexts.append(context)
                  train_questions.append(question)
                  train_answers.append(answer)
                elif random.randint(1,10) is 2:
                  val_contexts.append(context)
                  val_questions.append(question)
                  val_answers.append(answer)
    return train_contexts, train_questions, train_answers, val_contexts, val_questions, val_answers

train_contexts, train_questions, train_answers, val_contexts, val_questions, val_answers = read_squad() 

In [ ]:
# The following function is copied from https://huggingface.co/transformers/custom_datasets.html#question-answering-with-squad-2-0

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # Changed from original code to work for unanswerable questions. 
        if answer['text'] == '':
          answer['answer_start'] = 0
          answer['answer_end'] = 0
        else:
          if context[start_idx:end_idx] == gold_text:
              answer['answer_end'] = end_idx
          elif context[start_idx-1:end_idx-1] == gold_text:
              answer['answer_start'] = start_idx - 1
              answer['answer_end'] = end_idx - 1  
          elif context[start_idx-2:end_idx-2] == gold_text:
              answer['answer_start'] = start_idx - 2
              answer['answer_end'] = end_idx - 2   

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
# Design decision: using uncased instead of case, and Bert instead of DistilBert. 
config = BertConfig.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
# https://huggingface.co/transformers/custom_datasets.html#question-answering-with-squad-2-0

def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
      # Change Hugging Face code to work for unanswerable questions. 
      if answers[i]['text'] == '': 
        start_positions.append(0)
        end_positions.append(0)
      else:
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
# Now, the keys+values of the encoding are each of the inputs we need for the model.
# 1. input ids, 2. token type ids, 3. attention masks, 4. start positions of answers, 5. end positions. 
print(train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [ ]:
# Testing that I correctly changed unanswerables to have an answer index of 0 (previously nan which broke model during model.fit). 
import math
print(train_encodings['start_positions'][100:200])

# Making sure there are no infinities or NANs in the data. 
for i in train_encodings['start_positions']:
  if math.isinf(i) or math.isnan(i):
    print("here")
for i in train_encodings['end_positions']:
  if math.isinf(i) or math.isnan(i):
    print("here")

[191, 232, 55, 29, 40, 43, 215, 9, 1, 35, 159, 2, 8, 30, 102, 4, 6, 27, 22, 63, 51, 73, 51, 133, 23, 50, 10, 31, 7, 134, 59, 125, 26, 70, 54, 4, 66, 78, 88, 66, 17, 44, 12, 91, 64, 41, 15, 1, 110, 324, 175, 60, 11, 57, 118, 24, 0, 0, 9, 0, 120, 0, 154, 44, 0, 59, 0, 77, 29, 81, 30, 0, 135, 21, 80, 2, 32, 21, 10, 0, 0, 0, 0, 0, 56, 0, 0, 0, 9, 27, 0, 0, 118, 11, 18, 180, 0, 0, 0, 11]


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(({key: train_encodings[key] for key in ['input_ids', 'token_type_ids', 'attention_mask']},
                                                    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}))
val_dataset = tf.data.Dataset.from_tensor_slices(({key: val_encodings[key] for key in ['input_ids', 'token_type_ids', 'attention_mask']},
                                                  {key: val_encodings[key] for key in ['start_positions', 'end_positions']}))

In [ ]:
print(train_encodings['input_ids'][0])
print(train_encodings['token_type_ids'][0])
print(train_encodings['attention_mask'][0])
print(train_encodings['start_positions'][0])
print(train_encodings['end_positions'][0])

[101, 1037, 2969, 1011, 2649, 1000, 2715, 1011, 2154, 10469, 1000, 1010, 20773, 9005, 2774, 2008, 2024, 2411, 7356, 2011, 6991, 1997, 2293, 1010, 6550, 1010, 1998, 18847, 22864, 2100, 1010, 2004, 2092, 2004, 2931, 13798, 1998, 23011, 1012, 2006, 2754, 1010, 2014, 8790, 1010, 3811, 23317, 4616, 2031, 2419, 2000, 4401, 16889, 2075, 2014, 2004, 2028, 1997, 1996, 2190, 21751, 2015, 1999, 3824, 2759, 2189, 1012, 2802, 1037, 2476, 13912, 2539, 2086, 1010, 2016, 2038, 2853, 2058, 12963, 2454, 2636, 2004, 1037, 3948, 3063, 1010, 1998, 1037, 2582, 3438, 2454, 2007, 10461, 1005, 1055, 2775, 1010, 2437, 2014, 2028, 1997, 1996, 2190, 1011, 4855, 2189, 3324, 1997, 2035, 2051, 1012, 2016, 2038, 2180, 2322, 8922, 2982, 1998, 2003, 1996, 2087, 4222, 2450, 1999, 1996, 2400, 1005, 1055, 2381, 1012, 1996, 3405, 3068, 2523, 1997, 2637, 3858, 2014, 2004, 1996, 2327, 7378, 3063, 1999, 2637, 2076, 1996, 8876, 5476, 1012, 1999, 2268, 1010, 4908, 2315, 2014, 1996, 2327, 2557, 2774, 3063, 1997, 1996, 5476, 1010

In [ ]:
print(val_encodings['input_ids'][0])
print(val_encodings['token_type_ids'][0])
print(val_encodings['attention_mask'][0])
print(val_encodings['start_positions'][0])
print(val_encodings['end_positions'][0])

[101, 2012, 2287, 2809, 1010, 20773, 1998, 5593, 2767, 5163, 20539, 2777, 2474, 2696, 9035, 11111, 17753, 2096, 1999, 2019, 14597, 2005, 2019, 2035, 1011, 2611, 4024, 2177, 1012, 2027, 2020, 2872, 2046, 1037, 2177, 2007, 2093, 2060, 3057, 2004, 2611, 1005, 1055, 5939, 4168, 1010, 1998, 9680, 5669, 1998, 10948, 2006, 1996, 5848, 2265, 4984, 1999, 5395, 1012, 2044, 3773, 1996, 2177, 1010, 1054, 1004, 1038, 3135, 12098, 2638, 25312, 4590, 2716, 2068, 2000, 2010, 2642, 2662, 2996, 1998, 2872, 2068, 1999, 2732, 3945, 1010, 1996, 2922, 5848, 2265, 2006, 2120, 2694, 2012, 1996, 2051, 1012, 2611, 1005, 1055, 5939, 4168, 3478, 2000, 2663, 1010, 1998, 20773, 2101, 2056, 1996, 2299, 2027, 2864, 2001, 2025, 2204, 1012, 1999, 2786, 20773, 1005, 1055, 2269, 5295, 2013, 2010, 3105, 2000, 6133, 1996, 2177, 1012, 1996, 2693, 4359, 20773, 1005, 1055, 2155, 1005, 1055, 3318, 2011, 2431, 1010, 1998, 2014, 3008, 2020, 3140, 2000, 2693, 2046, 5459, 9620, 1012, 25436, 3013, 1996, 2434, 2240, 1011, 2039, 2000

In [ ]:
# It was difficult for me to change tf.data.Dataset.map() to fit these embeddings with additional inputs, so I
# hardcoded this replacement. Rather than creating all of the input layers using map(), I create each layer here.
input_ids = layers.Input(shape=(512,), dtype=tf.int32)
attention_mask = layers.Input(shape=(512,), dtype=tf.int32)
token_type_ids = layers.Input(shape=(512,), dtype=tf.int32)

In [ ]:
# Start by using our original TFBertModel to create first hidden layer. 
initlayer = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]

# Create two more layers and use a softmax for activation. 
hlayer1 = layers.Flatten()(layers.Dense(1, use_bias=False, name="hlayer1")(initlayer))
start = layers.Activation(keras.activations.softmax)(hlayer1)
hlayer2 = layers.Flatten()(layers.Dense(1, use_bias=False, name="hlayer2")(initlayer))
end = layers.Activation(keras.activations.softmax)(hlayer2)

In [ ]:
# Want final output to be probabilities generated using the two layers above. 
mymodel = keras.Model([input_ids, token_type_ids, attention_mask], [start, end])

In [ ]:
# Design decision: using SparseCategoricalCrossentropy because there are more than one input with labels.
# Using this instead of CategoricalCrossentropy to avoid a one-hot encoding.
# Documentation here: https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class .

optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits= False)
mymodel.compile(optimizer, loss)

In [ ]:
# TRAINING
input1 = [np.array(train_encodings['input_ids']), np.array(train_encodings['token_type_ids']), np.array(train_encodings['attention_mask'])]
input2 = [np.array(train_encodings['start_positions']), np.array(train_encodings['end_positions'])]

# VALIDATION
val1 = [np.array(val_encodings['input_ids']), np.array(val_encodings['token_type_ids']), np.array(val_encodings['attention_mask'])]
val2 = [np.array(val_encodings['start_positions']), np.array(val_encodings['end_positions'])]

In [ ]:
print("Example input IDs:")
print(input1[0][0]) 
print("Example token type IDs:")
print(input1[1][0]) 
print("Example attention mask:")
print(input1[2][0]) 

Example input IDs:
[  101  1037  2969  1011  2649  1000  2715  1011  2154 10469  1000  1010
 20773  9005  2774  2008  2024  2411  7356  2011  6991  1997  2293  1010
  6550  1010  1998 18847 22864  2100  1010  2004  2092  2004  2931 13798
  1998 23011  1012  2006  2754  1010  2014  8790  1010  3811 23317  4616
  2031  2419  2000  4401 16889  2075  2014  2004  2028  1997  1996  2190
 21751  2015  1999  3824  2759  2189  1012  2802  1037  2476 13912  2539
  2086  1010  2016  2038  2853  2058 12963  2454  2636  2004  1037  3948
  3063  1010  1998  1037  2582  3438  2454  2007 10461  1005  1055  2775
  1010  2437  2014  2028  1997  1996  2190  1011  4855  2189  3324  1997
  2035  2051  1012  2016  2038  2180  2322  8922  2982  1998  2003  1996
  2087  4222  2450  1999  1996  2400  1005  1055  2381  1012  1996  3405
  3068  2523  1997  2637  3858  2014  2004  1996  2327  7378  3063  1999
  2637  2076  1996  8876  5476  1012  1999  2268  1010  4908  2315  2014
  1996  2327  2557  2774  3063  

In [ ]:
mymodel.fit(input1, input2, batch_size=10, epochs=1, verbose=1, validation_data=[val1, val2])

In [ ]:
mymodel.save("bertmodel")

### Implementation Details

**Summary:**
I ultimately decided to use Bert and TensorFlow to create a question-answering model for SQuAD 2.0. There were several extenuating factors which went into this decision, as well as design decisions to convert these current packages to a model which supports answerability and detecting an answer span. 

**First Try:** 

*Implementation:*
When I first looked at this project, I wanted to implement my own version of Hugging Face's BertforQuestionAnswering model using PyTorch. (https://huggingface.co/transformers/model_doc/bert.html?highlight=bertforquestionanswering) My goal here was to use the baseline BertModel ("bert-base-uncased") and the functions provided by Hugging Face, but then customize that baseline model to take in additional inputs. The baseline BERT model utilizes input_ids and attention_masks, but to solve the problems of answerability and answer span, we would also need a way to identify which tokens in the question and context are part of the answer, and where that answer is located. Therefore, I added token_type_ids to the model, and start and end position arrays to map to answer span locations. In order to add these to the baseline model, I tokenized the context and question using BertTokenizer, passed it into a BertModel, then used a method similar to Project 3 where I ran nn.Linear for start positions and end positions of answer spans, then applied a softmax and calculated loss. 

*Why It Didn't Work:*
I ultimately (and unfortunately) decided to scrap this design for a few reasons. I ran out of CUDA memory on both my Cornell and personal email accounts. Even when I could run my model, RAM crashed on the first iteration of the neural network every single time, even when I was only inputting approximately 10% of the training data into it. I tried running it locally and uploading it in the hopes that would work, but I had trouble with the installation. I am sharing a house with 4 other people working from home, and my WiFi network is very slow as it is. 

**Second Try:** 

*Implementation:*
Even though most of my classmates on Piazza seem to be using PyTorch, I decided to try TensorFlow and see if that would work without crashing RAM and CUDA. This method worked and allowed my model to compile and fit the dataset. Below, I outline the steps in the code, and then I explain all of my design decisions. 

*Steps:*

1. Preprocessing the data: I discovered that the way I was processing data for my part 1 MEMM did not fit the exact requirements for Hugging Face and BERT, so I implemented a new method to iterate through the input json file and pull out necessary information. This time, I only needed to generate arrays for contexts, questions, and answers for both a training and validation set. Unlike the original BERT code, this preprocessing needs to take into account that some questions will not have an answer. This was at first causing NANs to be inputted into my model, since questions without an answer were being skipped over and None was used as the start and end positions for later inputs. To fix this, if a question is impossible I add an answer consisting of an empty string and a starting character of 0. To avoid RAM crashing, I had to use 1/10 of the data as a validation set and 1/5 as training. I know this is not ideal, but it was the only forseeable way for me to finish this project and get something to compile. I saw the suggestion in a Piazza post from another student. 

```
if (qa['is_impossible']):
  answer = {'text': '', 'answer_start': 0}
```

2. add_end_index(): This function was partially copied from Hugging Face to iterate through the contexts and answers we just generated, and add an ending character to each. To take into account the possibility of no answer, I added a simple boolean: if the answer is an empty string, then the ending character is 512, same as the starting. 

```
if answer['text'] == '':
  answer['answer_start'] = 0
  answer['answer_end'] = 0
```

3. Loading in BERT: I then created a configuration using BertConfig, tokenizer using BertTokenizerFast, and model using TFBertModel. All of these were implemented using "bert-base-uncased". 

4. Tokenizing the data: I used this tokenizer to creating encodings for the training and validation sets. 
5. Converting character positions to token positions: This function was also copied from Hugging Face and changed to include questions without answers. Basically, this function uses BertTokenizerFast's built-in method char_to_token, which converts a character position to the position of a single token within the tokenized data. 
```
start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
```
To work for unanswerables, I simply added an if statement checking if the answer was an empty string, and if so keeping the starting and ending positions the same, i.e. 0. This function also works for normalized data by checking to make sure that the indices are not slightly off. 

6. Creating the datasets: I now have an encoding as a dictionary with keys as the various inputs we need for our model. There are 5 inputs: input_ids, token_type_ids, attention_masks, starting_positions, and ending_positions. 
- Input IDs: These are nested arrays containing ints which map every token to an index number in the BERT tokenizer. 
- Token Type IDs: These are additional labels for every token, identifying it as either part of the context or question. 
- Attention Mask: This identifies whether or not every label is part of the actual token span, or is just padding. 
- Start Positions / End Positions: These are 1D arrays which give the starting positions and ending positions for answer span of every question+context. 
I also test here to make sure that none of the values in these sequences are NAN or INF, which is sometimes an explanation for a loss output of NAN. However, all of these labels are numbers within the correct range. 

7. I use tf.data.Dataset.from_tensor_slices to map the encodings to a dictionary, where each key is one of the 5 inputs listed above. I also print out examples of every input type to ensure that they look correct and are not all 0. 

8. Setting up the model: The next step in the Hugging Face documentation is to map the encodings/datasets above to layers which can be inputted into model.fit. I struggled to customize the lambda function to fit the extra 3 inputs, so I switched this to instead be 3 input layers with maximum dimension 512. I create the first hidden layer using the first row of my TFBertModel, and then create two additional layers using a softmax activation function. This is meant to replace the method I was using with PyTorch and CrossEntropyLoss, where I wanted to run start_positions and end_positions separately through nn.Linear, then apply softmax and calculate loss. I instantiate the final keras model, create a loss function using SparseCategoricalCrossentropy, an Adam optimizer, and compile the model. I have to pass that loss into the model twice because I ultimately have two inputs. 

9. The last step is to fit the model to the inputs and validation data, using 1 epoch and a batch size of 10. The inputs are two arrays: a nested array of input_ids, token_type_ids, and attention_masks, and another of starting and ending positons. I had to encode all of these are a np.array to avoid errors that the type was wrong. 
```
input1 = [np.array(train_encodings['input_ids']), np.array(train_encodings['token_type_ids']), np.array(train_encodings['attention_mask'])]
input2 = [np.array(train_encodings['start_positions']), np.array(train_encodings['end_positions'])]
```

**Design Decisions:** 

1. *bert-base-uncased vs. bert-base-cased:* I used the baseline BERT model with uncased words to avoid adding unnecessary length to all of the token arrays. For example, using cased would have separated the words "the" and "The", and I was already running out of memory. The one concern here is that removing uppercase letters could harm any named-entity or POS tagging that a model might be doing, but that didn't seem to be a problem here. Therefore, bert-base-uncased is the simpler and more efficient option. 
2. *Setting maximum length as 512:* I learned through trial-and-error that bert-base-uncased has a maximum input length of 512, so this is the obvious choice. 
3. *BertTokenizerFast vs. BertTokenizer:* I was initially just using BertTokenizer to tokenize all of the input data, but then I discovered in the Hugging Face documentation that BertTokenizerFast has a built-in function that converts character indices to token indices. I had already implemented one but was worried that it wouldn't work for this specific type of token normalization/tokenization, so I switched to this version. I kept my function in the code in case I needed it again (get_indices(set) and init_inputs(set)).
4. *Setting unanswerable positions as 0 vs. 512 vs. -1:* The trickiest part of converting BERT to handle unanswerable questions was deciding how to treat those answers. I ultimately decided to use an empty string instead of something like "No answer", in case this caused errors with indices in all of the input sequences. I initially tried to use -1 as the starting and ending positions, but this was incompatible with the BertTokenizer. I decided that 0, instead of something like 512 or the maximum length of the sequence, was the simplest/best option. 
5. *Using SparseCategoricalCrossentropy Loss:* I decided to use SparseCategoricalCrossentropy Loss after reading Keras/TensorFlow documentation (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class). Categorical losses were the best option for this dataset because there were multiple types/classes of labels being assigned to the tokens. Cross Entropy was the best option for Project 3, so I wanted to use it here again. It was also the closest match to CrossEntropyLoss() that I was previously using with PyTorch. Lastly, I used sparse instead of normal because I wanted to avoid one-hot encodings and the unnecessary space they consume. Also, one-hot encodings were a problem with my Project 3 because of vanishing gradients and leading to 0 learning, so SparseCategoricalCrossEntropy was the ultimate decision. 
6. *Using Adam with LR of 5e-5:* Adam was the best optimizer in my Project 3 for classifications with multiple label types, so I wanted to use it again here. In the article "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding", they ideal discussed learning rate and number of epochs and came to the conslusion that LR = 5e-5 was optimal for this learning task (https://arxiv.org/pdf/1810.04805.pdf). 
7. *Number of epochs and batch size:* There were a lot of constraints to this project, especially running out of GPU and RAM. I wanted to use a larger batch size, but unfortunately was unable to do to these constraints. I also wanted to use more epochs, but with my slow WiFi and the constraints of Colab this would have taken upwards of 4 hours (I tried). Therefore, I had to use 1 epoch and a batch_size of 10. I discuss in the next section why these are possible sources of error for my project. 








### Results Analysis


*Outputs During Training*
1. During the first iterations of epoch 1, the overall loss was 9.007, the loss for the first activation layer (predicting starting positions of answer spans) was 4.270, and the second activation layer (end positions) was 4.737. 
2. At the halfway point of training epoch 1, the overall loss was 5.935, first activation layer was 2.968, and for the second was 2.965. 

There was a definite overall loss change, as well as a loss change for both activation functions. If the loss function is changing by that much, then learning is occurring, and the model is doing its job. The inputs/model I created are working and correct. 

- This model is successful because it takes a holistic view of the problem, unlike the individual token/features MEMM. The inputs are various matrices which seem to be very successful for solving SQuAD 2.0 because they map every token to an ID in the vocabulary, whether or not it's in the answer, and whether or not it's in the question or context. Overall, this is what I was attempting to do with my MEMM, but this model is more successful because it is able to take in the entire context, while also taking into accoung inputs like attention masks and  functions like softmax activation/cross entropy loss. 
- This model is unsuccessful because of the nature of "bert-base-uncased". I chose this baseline BERT model because it is simple yet effective, has better tokenization than DistilBert, and leaves words uncased to take up less space ultimately. However, TFBertModel and bert-base-uncased have a maximum length of 512, which I learned by trial and error. This actually cuts off many of the inputs, which are questions concatenated with contexts. Therefore, a part of the context would be missing from many of the questions; if the correct answer span is in that cut-off portion, this would lead to questions being deemed unanswerable even though the answer lays outside of the max_length, or could even lead to the wrong output being produced if the model searches for the correct answer span in the minimized context. 

This system could be improved by:
1. Finding a BERT model which does not cut off questions and context at the maximum length of 512. 
2. Performing the model.fit calculation on the entirety of the SQuAD 2.0 dataset, rather than the 20% it is currently being trained on.
3. Using a larger batch size. This is actually a tremendous problem, and I predict it is one potential source of error as to why the loss function ended up being NAN. The batch size I had to use due to RAM and GPU constraints was extremely small, much smaller than most models performing these computations (10 instead of 64+). The small batch size led to increased computation times (maybe why Colab crashed); just training a single epoch took upwards of 1 hour before crashing.  
4. Using BertforQuestionAnswering, which has already been created for this exact dataset. 




#Comparing the two models

---

My MEMM is outputting fairly decent accuracies (~30%) because it succeeds in predicting answerability but fails to output the correct answer span. My BERT model, on the other hand, is not outputting anything. However, if I was able to successfully complete training, I believe this BERT model would have been more successful due to the changing loss, impact of learning, and its consideration of a more holistic/sequential view of every input. The loss was changing before crashing, so I know that the model was training on the data. 

The way data is normalized differs between my two models. For the MEMM, I removed punctuation but kept letter case to improve the ability of the POS  and named-entity taggers to preserve facets of tokens such as proper nouns. For the BERT model, I used an uncased version in order to save memory space and prevent the model from distinguishing between arbitrary words like "the" vs "The". This has an effect on the actual answer spans being outputted, causing the MEMM to be more successful at retaining information such as proper nouns and starting sentence tokens. 

I also encode unanswerable questions differently; the BERT model is more strict for things like input size and type, so I had to include the starting position of an answer span, even if it is not there, and encode it as 0 with length 0 for unanswerables. For MEMM, I was able to more flexibly encode features and tokens, thereby not constrained by things like maximum length and input type.  Therefore, the MEMM is more successful at detecting features of singular words, like their type and part-of-speech, and also has a better encoding for unanswerable questions due to its flexibility. 


### Error Analysis

1. I think that MEMM would be more successful with answerability for the same reason that it is worse at predicting exact answer span. The Markov Model is examining input sequences token by token; therefore, it was successful in detecting the absence of specific tokens with the correct named-entity and part of speech to match the question. For example, if the question asks "Where?", the MEMM is able to look token-by-token for any words in the context which match this part of speech (noun) and named-entity (location), and then consider probabilities outputted by the tokenized classifier for whether or not this should be the answer. Therefore, the MEMM model is successful at considering when a question should or should not have an answer given a specific context. The BERT model, on the other hand, takes a more much holistic view of the inputs, rather than viewing them as discrete tokens with features. However, it cuts contexts off at 512 (max length). This would impact answerability considerably; for questions for which the answer is after this point, it would deem the context unanswerable when it should not be. Also, because it is taking in concatenated questions and contexts and looking at sequences of specific word sequences, it would be worse at matching a specific named entity or word type to a question, unlike the MEMM. 
2. The BERT model would be more successful at actually predicting answer span, since it is a sequential model. It takes in matrices of input IDS, token types that indicate context/question, and start/end positions of actual answer spans. Rather than splitting all of these up into features per word token, and iterating through them one-by-one, it splits them up into features for the entire context+question sequence using an encoder-decoder structure and softmax. Due to this sequential nature and more holistic view of the inputs, it would ultimately be more successful than my token-based MEMM at outputting an actual answer sequence given a context. 




In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

!cp 'drive/My Drive/Colab Notebooks/Question-Answering on SQuAD 2.0 Dataset.ipynb' ./ 
!jupyter nbconvert --to PDF "Question-Answering on SQuAD 2.0 Dataset.ipynb"